# Satellite Trajectories
This notebook tutorial shows how to retrieve or create a trajectory in a variety of methods.
1. Retrieve a real trajectory through SSCWeb,
2. Create a sample trajectory in GDZ spherical coordinates,
3. Create a trajectory from a TLE input file,
4. Read in a trajectory from a file,
5. Write out a trajectory to a file.

## Retrieve a real satellite trajectory through SSCWeb 

In [1]:
# Import function to retrieve description.
from kamodo_ccmc.flythrough import SatelliteFlythrough as SF
help(SF.SatelliteTrajectory)

Help on function SatelliteTrajectory in module kamodo_ccmc.flythrough.SatelliteFlythrough:

SatelliteTrajectory(dataset, start_ts, stop_ts, coord_type='GEO', verbose=False)
    Retrieve and return satellite trajectory from HAPI/CDAWeb
    Parameters:
    ----------
    dataset: name of the satellite data set to pull trajectory from
    start_ts: utc timestamp for start of desired time interval
    stop_ts: utc timestamp for end of desired time interval
    coord_type: Pick from GEO, GSM, GSE, or SM
    verbose: Set to true to be overwhelmed with information.
    
    Coordinates are retrieved on a cartesian grid.
    See kamodo_ccmc.flythrough.utils.ConvertCoord for info on the coordinate
    systems.



C:\Users\rringuet\Anaconda3\envs\Kamodo_Jan2023\lib\site-packages\spacepy\time.py:2367: UserWarning: Leapseconds may be out of date. Use spacepy.toolbox.update(leapsecs=True)
  warnings.warn('Leapseconds may be out of date.'


In [2]:
# Retrieve a real satellite trajectory.
# Typical coordinates possible through SSCWeb are GEO, GSE, SM, and GSM (all cartesian and in R_E).
from datetime import datetime, timezone
start = datetime(2015, 3, 18, 6, 26, 40).replace(tzinfo=timezone.utc)
end = datetime(2015, 3, 18, 12, 11, 40).replace(tzinfo=timezone.utc)
traj_dict, coord_type = SF.SatelliteTrajectory('grace1', start.timestamp(), end.timestamp(), coord_type='GEO')
# Show the structure of the returned dictionary and range of each coordinate.
print(traj_dict.keys(), coord_type)
for key in traj_dict.keys():
    print(type(traj_dict[key]), traj_dict[key].shape, traj_dict[key].min(), traj_dict[key].max())

Attribute/Key names of return dictionary: dict_keys(['sat_time', 'c1', 'c2', 'c3'])
dict_keys(['sat_time', 'c1', 'c2', 'c3']) GEO-car
<class 'numpy.ndarray'> (345,) 1426660020.0 1426680660.0
<class 'numpy.ndarray'> (345,) -0.9270195119 1.0114874276
<class 'numpy.ndarray'> (345,) -1.0605838293 1.0499913722
<class 'numpy.ndarray'> (345,) -1.0639983632 1.0598825872


## Create a sample trajectory

In [3]:
# Another way to get a trajectory is via the SampleTrajectory flythrough function
help(SF.SampleTrajectory)

Help on function SampleTrajectory in module kamodo_ccmc.flythrough.SatelliteFlythrough:

SampleTrajectory(start_time, stop_time, max_lat=65.0, min_lat=-65.0, lon_perorbit=363.0, max_height=450.0, min_height=400.0, p=0.01, n=2.0)
    Given start and stop times in timestamp form, return a test satellite
    trajectory.
    Parameters:
    ----------
        start_time: utc timestamp in seconds for start
        stop_time: utc timestamp in seconds for stop
        max_lat: maximum latitude for sample trajectory, in degrees
            (default=65.)
        min_lat: minimum latitude for sample trajectory, in degrees
            (default=-65.)
        lon_perorbit: the degrees of longitude per about 90 minute orbit
            (set less than 360 for precession forward in longitude, set less
            than 360 for precession backwards) (default=363.)
        max_height: maximum starting height of orbit in km (default=450.)
        min_height: minimum starting height of orbit in km (default

In [4]:
# Create a sample trajectory.
sample_traj, sample_coord = SF.SampleTrajectory(start.timestamp(), end.timestamp(), max_height=550., 
                                                min_height=525., p=0.05, n=30.)
# Show the structure of the returned dictionary and range of each coordinate.
print(sample_traj.keys(), sample_coord)
for key in sample_traj.keys():
    print(type(sample_traj[key]), sample_traj[key].shape, sample_traj[key].min(), sample_traj[key].max())

Attribute/Key names of return dictionary: dict_keys(['sat_time', 'c1', 'c2', 'c3'])
(c1,c2,c3) = (lon, lat, alt) in (deg,deg,km) in the GDZ, sph coordinate system.
sat_time contains the utc timestamps.
dict_keys(['sat_time', 'c1', 'c2', 'c3']) GDZ-sph
<class 'numpy.ndarray'> (690,) 1426660000.0 1426680700.0
<class 'numpy.ndarray'> (690,) -180.0 179.48766328011612
<class 'numpy.ndarray'> (690,) -64.98998926658311 65.0
<class 'numpy.ndarray'> (690,) 499.26537612988614 548.3416630573417


## Create a trajectory from TLEs (two-line elements)

In [5]:
# TLE files can be obtained from celes-track.org and similar sites.
help(SF.TLETrajectory)

Help on function TLETrajectory in module kamodo_ccmc.flythrough.SatelliteFlythrough:

TLETrajectory(tle_file, start_utcts, stop_utcts, time_cadence, method='forward', verbose=False)
    Use sgp4 to calculate a satellite trajectory given TLEs.
    Parameters:
        tle_file: The file name, including complete file path, of a file
            containing two-line elements. It is assumed that the file has no
            header and no other content.
        start_utcts: The UTC timestamp corresponding to the desired start time.
            Should be an integer.
        stop_utcts: The UTC timestamp corresponding to the desired stop time.
            Should be an integer.
        time_cadence: The number of seconds desired between trajectory
            positions. Should be an integer.
        method: 'forward' or 'nearest'. This keyword changes the propagation
            method for timestamps between TLEs, not for timestamps before the
            first TLE or after the last TLE. The 'for

In [8]:
# How to get a trajectory from TLEs
tle_file = './Files/GRACE1_TLEs.txt'
time_cadence = 60.  # seconds between propagated trajectory positions
start_utcts = datetime(2015, 3, 18, 0, 20).replace(tzinfo=timezone.utc).timestamp()
end_utcts = datetime(2015, 3, 21, 0, 0).replace(tzinfo=timezone.utc).timestamp()
tle_dict, coord_type = SF.TLETrajectory(tle_file, start_utcts, end_utcts, time_cadence)
# Show the structure of the returned dictionary and range of each coordinate.
print(tle_dict.keys(), coord_type)
for key in tle_dict.keys():
    print(type(tle_dict[key]), tle_dict[key].shape, tle_dict[key].min(), tle_dict[key].max())

dict_keys(['sat_time', 'c1', 'c2', 'c3']) teme-car
<class 'numpy.ndarray'> (4301,) 1426638000.0 1426896000.0
<class 'numpy.ndarray'> (4301,) -0.210969381482701 0.21127247435341875
<class 'numpy.ndarray'> (4301,) -1.042345933737127 1.0437405655036243
<class 'numpy.ndarray'> (4301,) -1.0641529578213578 1.0598925509819095


## Load a trajectory from a file

In [9]:
help(SF.O.SF_read)

Help on function SF_read in module kamodo_ccmc.flythrough.SF_output:

SF_read(filename)
    Collect input function calls into one function.
    
    filename = string with complete filepath. The file extension must be one
        of 'nc'for a netCDF4 file, 'csv' for a comma separated file, or 'txt'
        for a tab separated file.
    
    Output: a nested dictionary containing the metadata, data, and units.



In [10]:
file_data = SF.O.SF_read('./Files/RealFlightExample_GITM.txt')
coord_sys = file_data['metadata']['coord_type'] + '-' + file_data['metadata']['coord_grid']
# Show the structure of the returned dictionary.
print(file_data.keys(), coord_sys)
for key in file_data.keys():
    print(file_data[key].keys())

dict_keys(['utc_time', 'c1', 'c2', 'c3', 'net_idx', 'rho_n', 'T_e', 'T_i', 'metadata']) GEO-car
dict_keys(['units', 'data'])
dict_keys(['units', 'data'])
dict_keys(['units', 'data'])
dict_keys(['units', 'data'])
dict_keys(['units', 'data'])
dict_keys(['units', 'data'])
dict_keys(['units', 'data'])
dict_keys(['units', 'data'])
dict_keys(['model_files', 'model_used', 'coord_type', 'coord_grid'])


## Write a trajectory to a file

In [11]:
help(SF.O.SF_write)

Help on function SF_write in module kamodo_ccmc.flythrough.SF_output:

SF_write(filename, model_filename, model_name, results_dict, results_units, coord_sys)
    Collect output function calls into one function.
    
    Inputs:
        filename = string with complete filepath. The file extension must be
            one of 'nc' for a netCDF4 file, 'csv' for a comma separated file,
            or 'txt' for a tab separated file.
        model_filename = A list of the model data filenames or prefixes used
            to generate the data. Filenames should include the full file path.
        model_name = A string indicating the model name.
        results_dict = A dictionary with variable names as keys (strings) and
            the time series data as the values (one array per key).
        results_units = A dictionary with variable names as keys (strings) and
            the units as the values (one value per key).
        coord_sys = one of 'GDZ', 'GEO', 'GSM', 'GSE', 'SM', 'GEI', 'MAG',


In [12]:
# Write grace1 TLE trajectory to a new file.
tle_units = {'sat_time': 's', 'c1': 'R_E', 'c2': 'R_E', 'c3': 'R_E'}
SF.O.SF_write('./Files/GRACE1_TLEs.csv', '', '', tle_dict, tle_units, 'teme-car')

'./Files/GRACE1_TLEs.csv'